# Urban dashboard

## Listes villes 

+ Cameroun
 + Yaoundé
 +    Douala
 + Bamenda
 + Bafoussam
 + Ngaoundéré
 + Nkongsamba
+ République Dominicaine
 +    Saint Domingue
+ Indonésie
 +    Bandung
+ Maroc
 + Fès
 + Rabat/ Salé
 + Oujda
 + Agadir
 + Kenitra
 + Beni Melal
 + Marrakech
 + Khourigba
 + Ej Jadida
 + Settat
 + Safi
 + Khenisset
 + Laayoune
 + Meknes
 + Tanger
 


### Phase 1: Get coordinates from a city list 

In [1]:
 maroc = ["fès", "rabat", "oujda", "agadir", "kenitra", "beni melal","marrakech","Khourigba","Ej Jadida","Settat","Safi","Khenisset","Laayoune","Meknes"," Tanger"]

cameroun = ["Yaoundé", "Douala", "Bamenda", "Bafoussam", "Ngaoundéré", "Nkongsamba"]

indonesie = ["Jakarta", "Surabaya", "Bandung"]

rep_dom = ["Saint Domingue"] 

liste_pays = {"Maroc":maroc, "Cameroun": cameroun, "Indonésie":indonesie, "République Dominicaine": rep_dom}

import geocoder


coo = {}
bbox = {}
for i in liste_pays:
    for j in liste_pays[i]:
        name= j+'_'+i
        g = geocoder.google(name)
        t =g.latlng
        coo[name] =  t
        bbox[name] = [t[0]-0.15, t[1]-0.15, t[0]+0.15, t[1]+0.15]
        


### Phase 2: Do OSM request and create corresponding JSON


In [3]:
import overpass
from string import Template


places_list = ['hospital', 'marketplace','uni','townhall']
api = overpass.API()
dict_query = {'hospital' : '''(
      node["amenity"="hospital"]$bbox;
      way["amenity"="hospital"]$bbox;
      relation["amenity"="hospital"]$bbox;
    )''' ,  'uni' : '''(
      node["amenity"="university"]$bbox;
      way["amenity"="university"]$bbox;
      relation["amenity"="university"]$bbox;
    )''' , 'townhall' : ''' (
      node["amenity"="townhall"]$bbox;
      way["amenity"="townhall"]$bbox;
      relation["amenity"="townhall"]$bbox;
    )
    ''' , 'marketplace' :'''
    ( node["amenity"="marketplace"]$bbox;
      way["amenity"="marketplace"]$bbox;
      relation["amenity"="marketplace"]$bbox;
    )
    '''}

for i in bbox:
    temp = str(bbox[i])
    
    bb = "("+temp[1:-1]+")"

    Input = {'bbox': bb}



    for z in dict_query:
        t = Template(dict_query[z]).safe_substitute(Input)
        print(t)
        response= api.Get(t)
        nam = i.replace(' ','')
        Input_save = {'query' : z, 'nam' : nam}
        name = Template('$nam-$query.geojson').safe_substitute(Input_save)
        print(name)
        #with open('data/'+name, "w") as f: f.write(str(response))


(
      node["amenity"="hospital"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      way["amenity"="hospital"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      relation["amenity"="hospital"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
    )
marrakech_Maroc-hospital.geojson

    ( node["amenity"="marketplace"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      way["amenity"="marketplace"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      relation["amenity"="marketplace"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
    )
    
marrakech_Maroc-marketplace.geojson
(
      node["amenity"="university"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      way["amenity"="university"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      relation["amenity"="university"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
    )
marrakech_Maroc-uni.geojson
 (
      node["amenity"="townhall"](31.4794723, -8.1310845, 31.7794723, -7.8310845);
      way["amenity"="townhal

### Phase 3: What if we create HTML files now ?

- Repasser par les json ...

In [22]:

html = ''' <!DOCTYPE html>
<html>

<head>
    <title> City Dashboard - $name</title>

    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

    <link rel="stylesheet" href="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.css" />
    <script src="https://npmcdn.com/leaflet@1.0.0-rc.3/dist/leaflet.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-ajax/2.1.0/leaflet.ajax.min.js"></script>


    <style>
        html,
        body {
            height: 100%;
        }
        
        #map {
            width: 100%;
            height: 100%;
        }
        
        .info {
            padding: 6px 8px;
            font: 14px/16px Arial, Helvetica, sans-serif;
            background: white;
            background: rgba(255, 255, 255, 0.8);
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
            border-radius: 5px;
        }
        
        .info h4 {
            margin: 0 0 5px;
            color: #777;
        }
        
        .legend {
            text-align: left;
            line-height: 18px;
            color: #555;
        }
        
        .legend i {
            width: 18px;
            height: 18px;
            float: left;
            margin-right: 8px;
            opacity: 0.7;
        }

    </style>
</head>

<body>

    <div id="map"></div>
    <script type="text/javascript">
        var map = L.map('map', {
            center: $center,
            zoom: 13,
            minZoom: 12
        });
        L.tileLayer('https://api.mapbox.com/styles/v1/etibdv/cj0kv2vnc003a2rt8m01gpf4h/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZXRpYmR2IiwiYSI6ImNpejhvdWJmcjAwMW8yd28weTkzMnA1aDkifQ.i8UKq0M_sIN1qq8F6UAgFw', {
            maxZoom: 18,
            attribution: 'Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
            '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
            'Dataviz from <a href="https://twitter.com/EtienneDavid">@EtienneDavid</a> with Python',
            id: 'mapbox.light'
        }).addTo(map);

        function onEachFeature(feature, layer) {
            var popupContent = "Type: " + feature.properties.amenity + "</br> Name: " +
            feature.properties.name;

            if (feature.properties && feature.properties.popupContent) {
                popupContent += feature.properties.popupContent;
            }

            layer.bindPopup(popupContent);
        }


        var style_hospital = {
            "color": "#f00000",
            "backgroundColor": "#f00000",
            "weight": 5,
            "opacity": 0.65
        };

        var hospitalIcon = L.icon({
            iconUrl: 'icon/hospital.png',
            iconSize: [32, 32],
            iconAnchor: [16, 37],
            popupAnchor: [0, -28]
        });

        var geojsonLayer_hospital = new L.GeoJSON.AJAX("data/$name-hospital.geojson", {
            style: style_hospital,
            onEachFeature: onEachFeature,

            pointToLayer: function(feature, latlng) {
                return L.marker(latlng, {
                    icon: hospitalIcon
                });
            }
        });
        geojsonLayer_hospital.addTo(map);

        var style_uni = {
            "color": "#12a81c",
            "weight": 5,
            "opacity": 0.65
        };
        
        var uniIcon = L.icon({
            iconUrl: 'icon/uni.png',
            iconSize: [32, 32],
            iconAnchor: [16, 37],
            popupAnchor: [0, -28]
        });

        var geojsonLayer_uni = new L.GeoJSON.AJAX("data/$name-uni.geojson", {
            onEachFeature: onEachFeature,
            style: style_uni,
                        pointToLayer: function(feature, latlng) {
                return L.marker(latlng, {
                    icon: uniIcon
                });
                }
        });


        geojsonLayer_uni.addTo(map);
   var townhallIcon = L.icon({
            iconUrl: 'icon/townhall.png',
            iconSize: [32, 32],
            iconAnchor: [16, 37],
            popupAnchor: [0, -28]
        });
        var geojsonLayer_townhall = new L.GeoJSON.AJAX("data/$name-townhall.geojson", {
            onEachFeature: onEachFeature,
                        pointToLayer: function(feature, latlng) {
                return L.marker(latlng, {
                    icon: townhallIcon
                });
                }
        });
        geojsonLayer_townhall.addTo(map)

        var style_marketplace = {
            "color": "#e9e223",
            "weight": 5,
            "opacity": 0.65
        };
        
           var marketIcon = L.icon({
            iconUrl: 'icon/market.png',
            iconSize: [32, 32],
            iconAnchor: [16, 37],
            popupAnchor: [0, -28]
        });

        var geojsonLayer_marketplace = new L.GeoJSON.AJAX("data/$name-marketplace.geojson", {
            onEachFeature: onEachFeature,
            style: style_marketplace,
            pointToLayer: function(feature, latlng) {
                return L.marker(latlng, {
                    icon: marketIcon
                });
                }
        });


        geojsonLayer_marketplace.addTo(map)

        L.control.scale({"imperial": false, "maxWidth" : 200}).addTo(map);
        console.log(map.getBounds());

    </script>
</body>

</html>'''

for i in coo:
    name =i.replace(' ','')
    Input = {'center' : coo[i], 'name' : name}
    maps = Template(html).safe_substitute(Input)
    with open(name+'.html', "w") as f: f.write(maps)


### Creating a small index



In [27]:
link=''
for i in coo:
    name =i.replace(' ','')
    print(name)
    maps = '<a href="'+name+'.html" target="search_iframe">'+i+'</a> '
    print(maps)
    link = link+'''    '''+maps 
    



html = '''<p>Select city below:</p>''' + link+'''

<iframe src="'''+name+'''.html" width="100%" height="100%" frameborder="0" name="search_iframe"></iframe>
'''

print(html)
html1 = ''' 



<!DOCTYPE html>
<html>
<head>
  <meta http-equiv="content-type" content="text/html; charset=UTF-8">
     <meta charset="utf-8">
   <link rel="stylesheet" type="text/css" href="http://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.4.0/css/font-awesome.min.css">
	<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Open+Sans"/>
<style type="text/css">

.menu {
	margin-left:15%;
	margin-top:0px;
	width:84%;
	height:8%;
	position: absolute;
}

.logo {
	margin-top:15px;
	margin-left:100px;
	width:7%;
	position: absolute;
}


ul {
    list-style-type: none;
    margin: 0;
    padding: 0;
    overflow: hidden;
    background-color: #EC635A;
	font-family: 'Today SB', 'Open Sans', 'Helvetica', arial, sans-serif;
	font-weight: bold;
	height: 60px;
	line-height: 32px;
	font-size:20px;
	color:white;
}

li {
    float: left; 
}


li a {
    display: block;
    color: #f9c3c0;
    text-align: center;
    padding: 14px 16px;
    text-decoration: none;
}

li a:hover:not(.active) {
    color: white;
}

#menu a:active {
     color:white;
}

.active {
    color: #fff;
	font-size:22px;
}
.about {
	background-color: #F0847D;
	float:right;
}

.mainframe {
	z-index:-2;
	width:98%;
	height:90%;
	margin-left:0px;
	margin-top:0px;
	position: absolute;

}

.frame_css {
    border : 0;
	z-index:1;
	width:95%;
	height:90%;
	margin-left:5%;
	margin-top:5%;
	position: relative;
}

.frameHome_css{
	z-index:1;
	width:95%;
	height:1400px;
	margin-left:5%;
	margin-top:7%;
	position: relative;
}
    
/**/

li a, .dropbtn {
    display: inline-block;
    color: white;
    text-align: center;
    padding: 14px 16px;
    text-decoration: none;
    font-family: 'Open Sans', 'Helvetica', arial, sans-serif;
	font-weight: bold;
	font-size:20px;
}

li a:hover, .dropdown:hover .dropbtn {
    background-color: #f7a35c;
}

li.dropdown {
    display: inline-block;
    
}

.dropdown-content {
    display: none;
    position: absolute;
    background-color: #f9f9f9;
    min-width: 160px;
    box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);
}

.dropdown-content a {
    color:  #423e3e;
    padding: 12px 16px;
    text-decoration: none;
    display: block;
    text-align: left;
    font-family:  'Open Sans', 'Helvetica', arial, sans-serif;
	font-weight: bold;
	font-size:16px;
}
}

.dropdown-content a:hover {background-color: #f1f1f1}

.dropdown:hover .dropdown-content {
    display: block;
}
 a, img {
    border:none !important ;
    outline:none !important ;
}   
  </style>
  
  <title>Portail des données @AFD_France - @Thomas_Roca</title>
<script>
  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');

  ga('create', 'UA-85673723-1', 'auto');
  ga('send', 'pageview');

</script>
  
<script type='text/javascript'>

	$(window).load(function(){
	
		$('#frame').html('<iframe src="home.html" width="100%" height="100%" frameborder="0"></iframe>');
		
		$("#Glance").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="Glance.html" width="95%" height="95%" frameborder="0"></iframe>');
		    $(this).addClass('active');
			$("#Inequality").removeClass('active');
			$("#Data").removeClass('active');
			$("#Country_Dashboard").removeClass('active');
			$("#home").removeClass('active');
			$("#local").removeClass('active');
            $("#BigBrowser").removeClass('active');
		});
		//$(this).addClass('active');

		$("#Inequality").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="inequalityDashboard.html" width="100%" height="100%" frameborder="0"></iframe>');
		   $(this).addClass('active');
		   	$("#Glance").removeClass('active');
			$("#Data").removeClass('active');
			$("#Country_Dashboard").removeClass('active');
			$("#about").removeClass('active');
			$("#home").removeClass('active');
			$("#local").removeClass('active');
            $("#BigBrowser").removeClass('active');
		});

		$("#home").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="home.html" width="100%" height="100%" frameborder="0"></iframe>');
		   $(this).addClass('active');
		   	$("#Glance").removeClass('active');
			$("#Data").removeClass('active');
			$("#Country_Dashboard").removeClass('active');
			$("#about").removeClass('active');
			$("#local").removeClass('active');
            $("#BigBrowser").removeClass('active');

		});

		$("#Data").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="data_code.html" width="100%" height="100%" frameborder="0"></iframe>');
		   $(this).addClass('active');
		   	$("#Glance").removeClass('active');
			$("#Country_Dashboard").removeClass('active');
			$("#Inequality").removeClass('active');
			$("#about").removeClass('active');
			$("#home").removeClass('active');
			$("#local").removeClass('active');
            $("#BigBrowser").removeClass('active');
		});

		$("#Country_Dashboard").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="Country_dashboard.html" width="100%" height="100%" frameborder="0"></iframe>');
		   $(this).addClass('active');
		   	$("#Glance").removeClass('active');
			$("#Data").removeClass('active');
			$("#Inequality").removeClass('active');
			$("#about").removeClass('active');
			$("#home").removeClass('active');
			$("#local").removeClass('active');
            $("#BigBrowser").removeClass('active');
		});
		
		$("#local").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="http://afd.countrydashboards.com/dataviz/local/Country_dashboard_local.html" width="100%" height="100%" frameborder="0"></iframe>');
		   $(this).addClass('active');
			$("#Country_Dashboard").removeClass('active');
		   	$("#Glance").removeClass('active');
			$("#Data").removeClass('active');
			$("#Inequality").removeClass('active');
			$("#about").removeClass('active');
			$("#home").removeClass('active');
            $("#BigBrowser").removeClass('active');
		});
        
        
        $("#Initiative_sahel").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="http://afd.countrydashboards.com/datastory/Initiative+Sahel.html" width="100%" height="100%" frameborder="0"></iframe>');})
        
		$("#WDI").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="http://afd.countrydashboards.com/WDI+browser.html" width="100%" height="100%" frameborder="0"></iframe>');})
        $("#DHS").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="http://afd.countrydashboards.com/DHS+browser.html" width="100%" height="100%" frameborder="0"></iframe>');})
        $("#FAO").click(function(e) {
		   e.preventDefault();
		   $('#frame').html('<iframe src="http://afd.countrydashboards.com/FAO+browser.html" width="100%" height="100%" frameborder="0"></iframe>');})
	});
</script>

</head>
<body>

<a href="http://www.afd.fr" target="_blank"><img class="logo" src="logo.jpg"><img/></a>
  
	<div class="menu">
	<ul>
	   <li><a href="" id="home" class="active"><i class="fa fa-home w3-xlarge"></i></a></li>
	  <li><a  href="Country_dashboard.html" id="Country_Dashboard">Country Dashboard </a></li>
	   <li><a href="Glance.html" id="Glance">At a Glance </a></li>
	   <li><a href="inequalityDashboard.html" id="Inequality">Inégalités</a></li>
	   <li><a href="/dataviz/local/Country_dashboard_local.html" id="local">Cartes DHS</a></li>
        
        <li class="dropdown">
        <a href="javascript:void(0)" class="dropbtn">#DataStory</a>
        <div class="dropdown-content">
            <a href="http://afd.countrydashboards.com/datastory/Initiative+Sahel.html" id="Initiative_sahel">Initiative Sahel - construction</a>
            <!--a href="#">Soon: - UNHCR Refugees </a>
            <a href="#">Soon! - ILO stat</a-->
        </div>
        </li>
        
        <li class="dropdown">
        <a href="javascript:void(0)" class="dropbtn">Big Browser</a>
        <div class="dropdown-content">
            <a href="http://afd.countrydashboards.com/WDI+browser.html" id="WDI">World Bank WDI</a>
            <a href="http://afd.countrydashboards.com/DHS+browser.html" id="DHS">Demographic Health Surveys</a>
            <a href="http://afd.countrydashboards.com/FAO+browser.html" id="FAO">FAO crop prices</a>
            <!--a href="#">Soon: - UNHCR Refugees </a>
            <a href="#">Soon! - ILO stat</a-->
        </div>
        </li>
	  <li style="float:right"><a  href="" id="Data">#DataDriven wiki</a></li>
	</ul>
	</div>

	<div class="mainframe">
	<div id="frame" class="frame_css">
        $html
	</div >
	</div>
  
</body>

</html>


'''

Input = {'html': html}
html2 = Template(html1).safe_substitute(Input)
with open('index.html', "w") as f: f.write(html2)


marrakech_Maroc
<a href="marrakech_Maroc.html" target="search_iframe">marrakech_Maroc</a> 
Douala_Cameroun
<a href="Douala_Cameroun.html" target="search_iframe">Douala_Cameroun</a> 
Safi_Maroc
<a href="Safi_Maroc.html" target="search_iframe">Safi_Maroc</a> 
Tanger_Maroc
<a href="Tanger_Maroc.html" target="search_iframe"> Tanger_Maroc</a> 
SaintDomingue_RépubliqueDominicaine
<a href="SaintDomingue_RépubliqueDominicaine.html" target="search_iframe">Saint Domingue_République Dominicaine</a> 
Bamenda_Cameroun
<a href="Bamenda_Cameroun.html" target="search_iframe">Bamenda_Cameroun</a> 
Meknes_Maroc
<a href="Meknes_Maroc.html" target="search_iframe">Meknes_Maroc</a> 
Settat_Maroc
<a href="Settat_Maroc.html" target="search_iframe">Settat_Maroc</a> 
Nkongsamba_Cameroun
<a href="Nkongsamba_Cameroun.html" target="search_iframe">Nkongsamba_Cameroun</a> 
Khenisset_Maroc
<a href="Khenisset_Maroc.html" target="search_iframe">Khenisset_Maroc</a> 
Bafoussam_Cameroun
<a href="Bafoussam_Cameroun.html" t